In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s3e1/sample_submission.csv
/kaggle/input/playground-series-s3e1/train.csv
/kaggle/input/playground-series-s3e1/test.csv


In [2]:

import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split


**LOAD DATA**

In [3]:
train = pd.read_csv('/kaggle/input/playground-series-s3e1/train.csv', index_col=0)
test = pd.read_csv('/kaggle/input/playground-series-s3e1/test.csv', index_col=0)

**EDA**

In [4]:
train.head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,MedHouseVal
id,,,,,,,,,
0,2.3859,15.0,3.827160,1.112100,1280.0,2.486989,34.60,-120.12,0.980
1,3.7188,17.0,6.013373,1.054217,1504.0,3.813084,38.69,-121.22,0.946
2,4.7750,27.0,6.535604,1.103175,1061.0,2.464602,34.71,-120.45,1.576
3,2.4138,16.0,3.350203,0.965432,1255.0,2.089286,32.66,-117.09,1.336
4,3.7500,52.0,4.284404,1.069246,1793.0,1.604790,37.80,-122.41,4.500


In [5]:
train.describe()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,MedHouseVal
count,37137.000000,37137.000000,37137.000000,37137.000000,37137.000000,37137.000000,37137.000000,37137.000000,37137.000000
mean,3.851029,26.057005,5.163124,1.062204,1660.778919,2.831243,35.570030,-119.554329,2.079751
std,1.803167,12.158221,1.206242,0.096490,1302.469608,2.702413,2.083179,1.974028,1.158571
min,0.499900,2.000000,0.851064,0.500000,3.000000,0.950000,32.550000,-124.350000,0.149990
25%,2.602300,17.000000,4.357522,1.020305,952.000000,2.394495,33.930000,-121.800000,1.208000
50%,3.515600,25.000000,5.068611,1.054545,1383.000000,2.744828,34.190000,-118.450000,1.808000
75%,4.699700,35.000000,5.858597,1.088825,1856.000000,3.125313,37.700000,-118.020000,2.660000
max,15.000100,52.000000,28.837607,5.873181,35682.000000,502.990610,41.950000,-114.550000,5.000010


In [6]:
train.shape

(37137, 9)

In [7]:
train.isna().sum()  #No missing values

MedInc         0
HouseAge       0
AveRooms       0
AveBedrms      0
Population     0
AveOccup       0
Latitude       0
Longitude      0
MedHouseVal    0
dtype: int64

**INCREASE DATA SET - CONCAT WITH ORIGINAL DATA**

In [8]:
features = ['MedInc', 'HouseAge', 'AveRooms', 'AveBedrms', 'Population', 'AveOccup', 'Latitude', 'Longitude']
target = 'MedHouseVal'

In [9]:
from sklearn.datasets import fetch_california_housing

original_data = fetch_california_housing()
original_data = pd.DataFrame(data=np.hstack([original_data['data'], original_data['target'].reshape(-1, 1)]), columns=train.columns)
train = pd.concat([train, original_data]).reset_index(drop=True)


In [10]:
train.head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,MedHouseVal
0,2.3859,15.0,3.827160,1.112100,1280.0,2.486989,34.60,-120.12,0.980
1,3.7188,17.0,6.013373,1.054217,1504.0,3.813084,38.69,-121.22,0.946
2,4.7750,27.0,6.535604,1.103175,1061.0,2.464602,34.71,-120.45,1.576
3,2.4138,16.0,3.350203,0.965432,1255.0,2.089286,32.66,-117.09,1.336
4,3.7500,52.0,4.284404,1.069246,1793.0,1.604790,37.80,-122.41,4.500


In [11]:
train.shape

(57777, 9)

Attribute info source: https://scikit-learn.org/stable/datasets/real_world.html#california-housing-dataset
* MedInc median income in block group
* HouseAge median house age in block group
* AveRooms average number of rooms per household
* AveBedrms average number of bedrooms per household
* Population block group population
* AveOccup average number of household members
* Latitude block group latitude
* Longitude block group longitude

In [12]:
test.head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude
id,,,,,,,,
37137,1.7062,35.0,4.966368,1.096539,1318.0,2.844411,39.75,-121.85
37138,1.3882,22.0,4.187035,1.098229,2296.0,3.180218,33.95,-118.29
37139,7.7197,21.0,7.129436,0.959276,1535.0,2.888889,33.61,-117.81
37140,4.6806,49.0,4.769697,1.048485,707.0,1.743590,34.17,-118.34
37141,3.1284,25.0,3.765306,1.081633,4716.0,2.003827,34.17,-118.29


In [13]:
test.shape

(24759, 8)

In [14]:
test.isna().sum() #No missing data

MedInc        0
HouseAge      0
AveRooms      0
AveBedrms     0
Population    0
AveOccup      0
Latitude      0
Longitude     0
dtype: int64

In [15]:
features = ['MedInc','HouseAge','AveRooms','AveBedrms','Population','AveOccup','Latitude','Longitude']
y = train['MedHouseVal']
X_train,X_test, y_train,y_test = train_test_split(train[features],train['MedHouseVal'],test_size=0.1)
X_train.shape,X_test.shape,y_train.shape,y_test.shape

((51999, 8), (5778, 8), (51999,), (5778,))

**USING LGBM REGRESSOR**

In [16]:
from lightgbm.sklearn import LGBMRegressor
import lightgbm as lgbm
from catboost import CatBoostRegressor
from sklearn.model_selection import KFold
N_ESTIMATORS = 100_000


clfs = []
rmses = []

params= {
 'lambda_l1': 2,
 'num_leaves': 90,
 'feature_fraction': 1,
 'bagging_fraction': 1,
 'bagging_freq': 4,
 'min_data_in_leaf': 105,
 'max_depth': 20,
}

kf = KFold(n_splits=10, random_state=0, shuffle=True)
for train_index, val_index in kf.split(train):
    X_train, X_val = train[features].loc[train_index], train[features].loc[val_index]
    y_train, y_val = train[target][train_index], train[target][val_index]
    
    clf = LGBMRegressor(learning_rate=0.02, n_estimators=N_ESTIMATORS, metric='rmse', **params)
    clf.fit(X_train.values, y_train, eval_set=[(X_val, y_val)], callbacks=[lgbm.early_stopping(90, verbose=True)])
    preds = clf.predict(X_val.values)
    
    clfs.append(clf)
    rmses.append(mean_squared_error(y_val, preds, squared=False))
print(f'mean RMSE across all folds: {np.mean(rmses)}')

[LightGBM] [Warning] lambda_l1 is set=2, reg_alpha=0.0 will be ignored. Current value: lambda_l1=2
[LightGBM] [Warning] feature_fraction is set=1, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=1
[LightGBM] [Warning] bagging_freq is set=4, subsample_freq=0 will be ignored. Current value: bagging_freq=4
[LightGBM] [Warning] min_data_in_leaf is set=105, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=105
[LightGBM] [Warning] bagging_fraction is set=1, subsample=1.0 will be ignored. Current value: bagging_fraction=1
Training until validation scores don't improve for 90 rounds
Early stopping, best iteration is:
[1108]	valid_0's rmse: 0.516425
[LightGBM] [Warning] lambda_l1 is set=2, reg_alpha=0.0 will be ignored. Current value: lambda_l1=2
[LightGBM] [Warning] feature_fraction is set=1, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=1
[LightGBM] [Warning] bagging_freq is set=4, subsample_freq=0 will be ignored. Current val

**USING CATBOOST**

In [17]:
rmses = []
kf = KFold(n_splits=10, random_state=1, shuffle=True)
for train_index, val_index in kf.split(train):
    X_train, X_val = train[features].loc[train_index], train[features].loc[val_index]
    y_train, y_val = train[target][train_index], train[target][val_index]

    clf = CatBoostRegressor(iterations=N_ESTIMATORS, loss_function='RMSE')
    clf.fit(X_train, y_train, eval_set=(X_val, y_val), early_stopping_rounds=1000, verbose=False)
    
    
    preds = clf.predict(X_val.values)
    
    clfs.append(clf)
    rmses.append(mean_squared_error(y_val, preds, squared=False))
print(f'mean RMSE across all folds: {np.mean(rmses)}')

mean RMSE across all folds: 0.5200769664943921


In [18]:
test_preds = []

for clf in clfs:
    preds = clf.predict(test[features].values)
    test_preds.append(preds)

In [19]:
test_preds = np.stack(test_preds).mean(0)
test_preds

array([0.64171085, 1.02633995, 3.99365512, ..., 1.19639896, 3.65709978,
       3.51486165])

In [20]:
submission = pd.DataFrame({'id': test.index, 'MedHouseVal': test_preds})
submission.head()

,id,MedHouseVal
0,37137,0.641711
1,37138,1.026340
2,37139,3.993655
3,37140,3.507909
4,37141,2.414486


In [21]:
submission.to_csv('submission.csv', index=False)